In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import re
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException


d = webdriver.Chrome('C:/Users/Ina Lee/Downloads/chromedriver_win32/chromedriver')
d.implicitly_wait(3)

song_id_list = []
title_list = []
artist_list = []
lyric_list  = []
genre_list = []
date_list = []

for i in range(5050,8000,50): # 링크에 들어갈 startIndex 파라미터, 1부터 n번까지 50 간격으로
    song_list = []
    d.get("https://www.melon.com/genre/song_list.htm?gnrCode=GN0200#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="+str(i))
    sleep(3)
    for j in range(1,51): #한 페이지에 50곡씩
        song_ids = WebDriverWait(d, 20).until(lambda d : d.find_elements_by_xpath('//*[@id="frm"]/div/table/tbody/tr['+str(j)+']/td[4]/div/a'))
        try:
            song_ids = [re.sub('[^0-9]', '', song_id.get_attribute("href")) for song_id in song_ids]
        except StaleElementReferenceException:
            print("에러에러에러~!!!!!!")
            song_ids = WebDriverWait(d, 20).until(lambda d : d.find_elements_by_xpath('//*[@id="frm"]/div/table/tbody/tr['+str(j)+']/td[4]/div/a'))
            song_ids = [re.sub('[^0-9]', '', song_id.get_attribute("href")) for song_id in song_ids]
            pass
            
        song_list.append(song_ids) #한 페이지에 있는 곡에 대한 곡 id list에 담아줌

    for song_id in song_list: # 해당 곡 id 대입해서 상세 페이지 찾아가기
        d.get("https://www.melon.com/song/detail.htm?songId=" + song_id[0])
        sleep(3)
        html = d.page_source
        soup = BeautifulSoup(html, "html.parser")  
        
        ## 가사 등록 / 미등록 여부 확인.
        if (soup.find(attrs={"class": "lyric_none"}) == None)&(len(soup.find(attrs={"class": "wrap_lyric"}).text)>10): 
            pass
        else:
            print("가사 미등록")
            continue

        title = soup.find(attrs={"class": "song_name"}).text.replace('곡명', '')

        if '19금' in title:
            title = title.replace('19금', '')

        title = title.replace("\n","").replace("\t","")
        title_list.append(title)

        print(soup.select('#downloadfrm > div > div > div.entry > div.info > div.artist')[0].text.replace("\n",""))
        
        if "Various Artists" in soup.select('#downloadfrm > div > div > div.entry > div.info > div.artist')[0].text:
            artist = "Various Artists"
        else:
            artist = soup.find(attrs={"class": "artist_name"}).text
        
        artist_list.append(artist)

        genre = soup.select('#downloadfrm > div > div > div.entry > div.meta > dl > dd')[2].text
        genre_list.append(genre)

        lyric = str(soup.select("#d_video_summary")[0])
        lyric = lyric.replace('<div class="lyric" id="d_video_summary">','').replace('</div>','').replace('<br/>',' ')
        lyric_list.append(lyric)

        date = soup.select('#downloadfrm > div > div > div.entry > div.meta > dl > dd')[1].text
        date_list.append(date)
        
        song_id_list.append(song_id[0])

        print("곡 id:", song_id[0])
        print("제목:", title)
        print("아티스트:", artist) 
        print("==============================")

        # list로 담아준 곡 정보들 dict로 만듦.
        result = {
            'song_id': song_id_list,
            'title': title_list,
            'artist': artist_list,
            'lyric': lyric_list,
            'date': date_list,
            'genre': genre_list
            }   

        
    #엑셀로 작성
    df_track_info = pd.DataFrame.from_dict(result, orient='index')
    df_track_info = df_track_info.transpose()
    writer = pd.ExcelWriter("댄스3.xlsx")
    df_track_info.to_excel(writer, sheet_name='댄스', startrow=1, header=True)
    writer.save()

여자친구 (GFRIEND)
곡 id: 30630740
제목: LIFE IS A PARTY
아티스트: 여자친구 (GFRIEND)
TWICE (트와이스)
곡 id: 30781488
제목: DING DONG
아티스트: TWICE (트와이스)
승리
곡 id: 31199876
제목: 몰라도 (MOLLADO) (Feat. B.I)
아티스트: 승리
김연지
곡 id: 31749804
제목: Victory
아티스트: 김연지
영턱스클럽
곡 id: 74258
제목: Love Designer
아티스트: 영턱스클럽
KYT (코요태)
곡 id: 432433
제목: 실연
아티스트: KYT (코요태)
KYT (코요태)
곡 id: 454499
제목: 단념
아티스트: KYT (코요태)
KYT (코요태)
곡 id: 500969
제목: 아리랑 목동
아티스트: KYT (코요태)
쿨 (COOL)
곡 id: 521734
제목: 십계(십계명)
아티스트: 쿨 (COOL)
SS501
곡 id: 1030700
제목: 하얀 사람
아티스트: SS501
소찬휘
곡 id: 1741989
제목: Tears
아티스트: 소찬휘
신화
곡 id: 3745238
제목: Stay
아티스트: 신화
니엘
곡 id: 30204049
제목: 날 울리지마 (Feat. 자이언트핑크)
아티스트: 니엘
쿨 (COOL)
곡 id: 27231
제목: Love Letter
아티스트: 쿨 (COOL)
쿠키
곡 id: 27792
제목: 초애(初愛) (Remix)
아티스트: 쿠키
김범룡
곡 id: 862088
제목: 바람바람바람
아티스트: 김범룡
보아 (BoA)
곡 id: 2795648
제목: Dangerous
아티스트: 보아 (BoA)
스피드
곡 id: 4012509
제목: IT`S OVER (Feat. 박보영)
아티스트: 스피드
C-CLOWN
곡 id: 4088051
제목: 흔들리고 있어
아티스트: C-CLOWN
SHINee (샤이니)
곡 id: 4274233
제목: 1분만 (One Minute Back)
아티스트: SHINee (샤이니)
인피니트

서인국, DK
곡 id: 4458957
제목: 슬픈거짓말
아티스트: 서인국
VIXX (빅스)
곡 id: 4660469
제목: Love, LaLaLa
아티스트: VIXX (빅스)
김정은
곡 id: 5417805
제목: 프로포즈
아티스트: 김정은
TWICE (트와이스)
곡 id: 30781493
제목: 힘내! (DON`T GIVE UP)
아티스트: TWICE (트와이스)
모모랜드 (MOMOLAND)
곡 id: 31149420
제목: 빙고게임
아티스트: 모모랜드 (MOMOLAND)
TWICE (트와이스)
곡 id: 31487799
제목: YOUNG & WILD
아티스트: TWICE (트와이스)
티티마
곡 id: 1776010
제목: 메리 메리
아티스트: 티티마
SHINee (샤이니)
곡 id: 1920700
제목: 내 곁에만 있어 (Best Place)
아티스트: SHINee (샤이니)
엠블랙
곡 id: 3639793
제목: White Forever
아티스트: 엠블랙
헬로비너스
곡 id: 5789572
제목: 난 예술이야 (Club Remix)
아티스트: 헬로비너스
비스트
곡 id: 5794999
제목: 가져가
아티스트: 비스트
업텐션 (UP10TION)
곡 id: 7951976
제목: 나두 (dito)
아티스트: 업텐션 (UP10TION)
신우, 솔비
곡 id: 9647326
제목: 경고 (내가 스타 랍스타, 노량진 인어공주)
아티스트: 신우
TWICE (트와이스)
곡 id: 31487800
제목: SUNSET
아티스트: TWICE (트와이스)
체리블렛 (Cherry Bullet)
곡 id: 31818046
제목: 발그레 (Ruddy)
아티스트: 체리블렛 (Cherry Bullet)
공원소녀 (Girls in the Park)
곡 id: 31943380
제목: Black H●le
아티스트: 공원소녀 (Girls in the Park)
젝스키스
곡 id: 83806
제목: 기억해 줄래 (Remix)
아티스트: 젝스키스
쿨 (COOL)
곡 id: 521720
제목: 

씨야
곡 id: 1832985
제목: 바보
아티스트: 씨야
김종민
곡 id: 3799592
제목: 두근두근 (Feat. 길미)
아티스트: 김종민
Apink (에이핑크)
곡 id: 5781110
제목: I DO
아티스트: Apink (에이핑크)
비타민
곡 id: 7877035
제목: Happy Day
아티스트: 비타민
플레이백
곡 id: 30698668
제목: 말해줘
아티스트: 플레이백
청하
곡 id: 30844117
제목: Offset
아티스트: 청하
가사 미등록
나몰라패밀리 (Namolla Family)
곡 id: 1701605
제목: 사랑은 (Feat. 나비, 허니패밀리 주라)
아티스트: 나몰라패밀리 (Namolla Family)
SUPER JUNIOR (슈퍼주니어)
곡 id: 1759303
제목: 아주 먼 옛날 (Song For You) (Bonus Track)
아티스트: SUPER JUNIOR (슈퍼주니어)
캔
곡 id: 1944689
제목: 날보러와요
아티스트: 캔
김건모
곡 id: 2183145
제목: 넌 친구 난 연인 (Club House Re-Mix Edition)
아티스트: 김건모
장우혁
곡 id: 2605917
제목: Rain Man (Feat. 강현정 Of 버블시스터즈)
아티스트: 장우혁
틴탑
곡 id: 3666279
제목: Girl Friend
아티스트: 틴탑
투엑스
곡 id: 3883254
제목: 반지자국
아티스트: 투엑스
B1A4
곡 id: 4105657
제목: 몇 번을
아티스트: B1A4
미쓰에이
곡 id: 4308017
제목: Touch
아티스트: 미쓰에이
울랄라세션
곡 id: 5619413
제목: 풍문으로 들었소
아티스트: 울랄라세션
토르
곡 id: 7977055
제목: 크리스마스 노래
아티스트: 토르
에이프릴 (APRIL)
곡 id: 30183397
제목: 시간아 멈춰라
아티스트: 에이프릴 (APRIL)
엘리스 (ELRIS)
곡 id: 30621600
제목: Midnight, moonlight
아티스트: 엘리스 (ELRIS)
이

소녀시대-태티서 (Girls` Generation-TTS)
곡 id: 3786996
제목: 안녕 (Good-bye, Hello)
아티스트: 소녀시대-태티서 (Girls` Generation-TTS)
XIA (준수)
곡 id: 4172977
제목: Turn It Up (Feat. Dok2)
아티스트: XIA (준수)
B1A4
곡 id: 4393163
제목: Amazing
아티스트: B1A4
Lim Kim
곡 id: 5678245
제목: No more
아티스트: Lim Kim
보아 (BoA)
곡 id: 5692636
제목: Smash
아티스트: 보아 (BoA)
알리 (ALi), 치타 (CHEETAH)
곡 id: 30566476
제목: Don`t Speak
아티스트: 알리 (ALi)
Glanz2d (글란즈이디)
곡 id: 30843089
제목: See the Light (Feat. 인에이틴 수빈)
아티스트: Glanz2d (글란즈이디)
SUPER JUNIOR (슈퍼주니어)
곡 id: 1047728
제목: 오늘만은 (I`m Your Man)
아티스트: SUPER JUNIOR (슈퍼주니어)
뉴이스트
곡 id: 4734439
제목: Judgement
아티스트: 뉴이스트
DIA (다이아)
곡 id: 30374832
제목: 남.사.친
아티스트: DIA (다이아)
벅
곡 id: 71866
제목: 맨발의 청춘 (Remix)
아티스트: 벅
애플잼
곡 id: 1029078
제목: 크리스마스의 기억
아티스트: 애플잼
타이푼
곡 id: 1316305
제목: 토요일 밤 (Remix ver.)
아티스트: 타이푼
인순이
곡 id: 1336658
제목: 열정 (Origianl Ver.) (Feat. 미쓰라 진, 이블 몽키)
아티스트: 인순이
배슬기
곡 id: 1522852
제목: One By One
아티스트: 배슬기
G-DRAGON
곡 id: 2626714
제목: This Love (G.H Remix)
아티스트: G-DRAGON
블락비 (Block B)
곡 id: 3453991
제목: Wan

업텐션 (UP10TION)
곡 id: 30499673
제목: 매일 (Everyday)
아티스트: 업텐션 (UP10TION)
비타민
곡 id: 30778986
제목: 성탄절의 비밀소원
아티스트: 비타민
이지형
곡 id: 31694424
제목: 사뿐사뿐
아티스트: 이지형
SuperM
곡 id: 32079964
제목: I Can’t Stand The Rain (Inst.)
아티스트: SuperM
BVNDIT (밴디트)
곡 id: 32149274
제목: 드라마틱 (Dramatic)
아티스트: BVNDIT (밴디트)
JY
곡 id: 32162760
제목: Freedom
아티스트: JY
리미트리스 (LIMITLESS)
곡 id: 32219949
제목: LIMITLESS-INTRO
아티스트: 리미트리스 (LIMITLESS)
KYT (코요태)
곡 id: 314269
제목: 애련
아티스트: KYT (코요태)
거북이
곡 id: 1455711
제목: 칵테일 사랑 (Madteam Remix)
아티스트: 거북이
터보
곡 id: 1606456
제목: 검은 고양이 네로 (X-Mas Remix)
아티스트: 터보
거미
곡 id: 3648875
제목: 날 떠나지마 (원곡가수 박진영)
아티스트: 거미
디유닛
곡 id: 3868795
제목: I`m Missin` You
아티스트: 디유닛
에일리
곡 id: 4092533
제목: 여인의 향기
아티스트: 에일리
이효리
곡 id: 4118325
제목: Love Radar (Feat. 빈지노)
아티스트: 이효리
베리굿 (Berry Good)
곡 id: 5578739
제목: 요즘 너 때문에 난
아티스트: 베리굿 (Berry Good)
김완선
곡 id: 7916277
제목: 탤런트 (Rap By 서태원, 김민기)
아티스트: 김완선
여자친구 (GFRIEND)
곡 id: 8028706
제목: 시간을 달려서 (Inst.)
아티스트: 여자친구 (GFRIEND)
AOA
곡 id: 8181758
제목: Crazy Boy
아티스트: AOA
보이프렌드
곡 id: 81936

...

블락비 (Block B)
곡 id: 3349891
제목: Wanna B
아티스트: 블락비 (Block B)
2PM
곡 id: 3448190
제목: Hot
아티스트: 2PM
B1A4
곡 id: 3734725
제목: SUPER SONIC
아티스트: B1A4
신화
곡 id: 3745235
제목: Red Carpet
아티스트: 신화
문희준
곡 id: 4015105
제목: I`m not OK
아티스트: 문희준
용준형, 비투비, 하연수
곡 id: 4117604
제목: 지난 날
아티스트: 용준형
VIXX (빅스)
곡 id: 4191450
제목: CHAOS
아티스트: VIXX (빅스)
SHINee (샤이니)
곡 id: 4200508
제목: Girls, Girls, Girls
아티스트: SHINee (샤이니)
SHINee (샤이니)
곡 id: 4200512
제목: Runaway
아티스트: SHINee (샤이니)
엠블랙
곡 id: 4203062
제목: I Don`t Know
아티스트: 엠블랙
티아라
곡 id: 4271837
제목: 넘버나인 (No.9) (Club Ver.)
아티스트: 티아라
인피니트
곡 id: 4649351
제목: 미치겠어 (Infinite F)
아티스트: 인피니트
하이디
곡 id: 5652825
제목: 진이 (2015 Remake)
아티스트: 하이디
가사 미등록
맵식스 (MAP6)
곡 id: 7924490
제목: 휘파람
아티스트: 맵식스 (MAP6)
서인영
곡 id: 7933662
제목: 소리질러
아티스트: 서인영
쿨요태
곡 id: 8049265
제목: No!Mantic
아티스트: 쿨요태
박지헌
곡 id: 8094565
제목: 오늘도 난 (꼭두각시 인형 피노키오)
아티스트: 박지헌
바다 (BADA)
곡 id: 8222314
제목: AMAZING
아티스트: 바다 (BADA)
클래지콰이
곡 id: 9648442
제목: 잠 못드는 밤
아티스트: 클래지콰이
모모랜드 (MOMOLAND)
곡 id: 30087564
제목: Welcome to MOMOLAND
아티스트: 모

SF9
곡 id: 30166011
제목: 너와 함께라면 (So Beautiful) (Inst.)
아티스트: SF9
샤넌
곡 id: 30337742
제목: 난 여자가 있는데
아티스트: 샤넌
Red Queen
곡 id: 30425225
제목: ACACIA
아티스트: Red Queen
DIA (다이아)
곡 id: 30592680
제목: LO OK (빈챈현스S Ver.)
아티스트: DIA (다이아)
이달의 소녀 오드아이써클
곡 id: 30642276
제목: ODD
아티스트: 이달의 소녀 오드아이써클
다원 (우주소녀)
곡 id: 30710469
제목: Go Ready Go
아티스트: 다원 (우주소녀)
SUPER JUNIOR (슈퍼주니어)
곡 id: 30712777
제목: Spin Up!
아티스트: SUPER JUNIOR (슈퍼주니어)
IN2IT (인투잇)
곡 id: 31030849
제목: SnapShot
아티스트: IN2IT (인투잇)
틴탑
곡 id: 31062771
제목: 놀면 돼 (Let`s play!)
아티스트: 틴탑
마이틴
곡 id: 31177668
제목: Pretty in Pink
아티스트: 마이틴
빅플로
곡 id: 31252809
제목: 니 친구
아티스트: 빅플로
원어스(ONEUS)
곡 id: 31548077
제목: Intro : LIGHT US
아티스트: 원어스(ONEUS)
여고생 (High School)
곡 id: 31768326
제목: Baby You`re Mine (베유마)
아티스트: 여고생 (High School)
W24
곡 id: 31889932
제목: 솔파미레도 Solfamiredo
아티스트: W24
민세영
곡 id: 31935949
제목: 겟 아웃 (Get Out)
아티스트: 민세영
위시걸스 (Wish Girls)
곡 id: 31999287
제목: Yours is mine (넌내꺼)
아티스트: 위시걸스 (Wish Girls)
파안 (pa.an)
곡 id: 32132570
제목: SUNSHINE
아티스트: 파안 (pa.an)
서태지와 아이들
곡 

보아 (BoA)
곡 id: 316785
제목: Feelings Deep Inside (마음은 전해진다)
아티스트: 보아 (BoA)
신화
곡 id: 418469
제목: ...후에
아티스트: 신화
터보
곡 id: 497126
제목: X-Mas Dance Party (Radio Ver.)
아티스트: 터보
3Boys
곡 id: 520082
제목: 바람둥이
아티스트: 3Boys
사라
곡 id: 526541
제목: 나비
아티스트: 사라
보아 (BoA)
곡 id: 528534
제목: 두근두근 (Pit-A-Pat)
아티스트: 보아 (BoA)
신화
곡 id: 836973
제목: Hey, Dude! (코카콜라 CF 신화편)
아티스트: 신화
솔리드
곡 id: 987673
제목: 잠든 널 포켓 속에
아티스트: 솔리드
터보
곡 id: 1606521
제목: 검은 고양이 (Dance Mega Mix Ver.)
아티스트: 터보
터보
곡 id: 1606522
제목: 금지된 장난 (Dance Mega Mix Ver.)
아티스트: 터보
김혜림
곡 id: 1620596
제목: 이젠 떠나가 볼까 (Remix)
아티스트: 김혜림
백지영
곡 id: 1698265
제목: Sad Salsa
아티스트: 백지영
타이푼
곡 id: 1714586
제목: 설레임 (Ed)
아티스트: 타이푼
노홍철
곡 id: 1715767
제목: 소녀
아티스트: 노홍철
하이디
곡 id: 1741997
제목: 진이
아티스트: 하이디
양혜승
곡 id: 1742012
제목: 랑데뷰
아티스트: 양혜승
구피
곡 id: 1742013
제목: 게임의 법칙
아티스트: 구피
KYT (코요태)
곡 id: 1748376
제목: 환희
아티스트: KYT (코요태)
SUPER JUNIOR (슈퍼주니어)
곡 id: 1759288
제목: 소원이 있나요 (Sapphire Blue)
아티스트: SUPER JUNIOR (슈퍼주니어)
소녀시대 (GIRLS` GENERATION)
곡 id: 1766891
제목: Touch The Sky (Original Ver.)
아티

KYT (코요태)
곡 id: 1136334
제목: 상상 (영화 `청춘만화` 삽입곡)
아티스트: KYT (코요태)
조혜련
곡 id: 1265756
제목: 가라
아티스트: 조혜련
Placebo Effect
곡 id: 1368919
제목: Love Effect
아티스트: Placebo Effect
키로츠
곡 id: 1506268
제목: 신기한 개구리 케이크
아티스트: 키로츠
SAT
곡 id: 1570690
제목: 미녀와 야수 (Ok? Ok!)
아티스트: SAT
터보
곡 id: 1606634
제목: 왜 몰라
아티스트: 터보
김범룡
곡 id: 1643405
제목: 바람, 바람, 바람
아티스트: 김범룡
바이브
곡 id: 1682905
제목: 그남자 그여자 (Remix Ver.)
아티스트: 바이브
백지영
곡 id: 1698240
제목: Sad Salsa
아티스트: 백지영
빅마마
곡 id: 1742134
제목: 천국 (Feat. MC 몽)
아티스트: 빅마마
KYT (코요태)
곡 id: 1748382
제목: 캐롤메들리 Feliznavidad
아티스트: KYT (코요태)
초신성
곡 id: 1755469
제목: Hit (히트)
아티스트: 초신성
타이푼
곡 id: 1778658
제목: 기다릴게
아티스트: 타이푼
베이비 복스
곡 id: 1903291
제목: Why
아티스트: 베이비 복스
태인
곡 id: 1914577
제목: I Love You
아티스트: 태인
손담비
곡 id: 1964748
제목: 미쳤어 Remix (Club Hiphop Ver.)
아티스트: 손담비
바다 (BADA)
곡 id: 2245549
제목: Yes I`m In Love (Feat. 2PM 택연)
아티스트: 바다 (BADA)
가비엔제이
곡 id: 2301645
제목: 이별주의보
아티스트: 가비엔제이
바다 (BADA)
곡 id: 2314862
제목: Yes I`m In Love (Feat. 2PM 택연)
아티스트: 바다 (BADA)
일렉트로보이즈
곡 id: 2705251
제목: 전화가 오네 (Feat. 호란 Fr

김태우
곡 id: 2276197
제목: 기억과 추억 (Feat. 준형,호영,데니)
아티스트: 김태우
f(x)
곡 id: 2670465
제목: Me+U
아티스트: f(x)
SHINee (샤이니)
곡 id: 2952168
제목: A-Yo
아티스트: SHINee (샤이니)
SHINee (샤이니)
곡 id: 2952177
제목: 사.계.후 (Love Still Goes On)
아티스트: SHINee (샤이니)
가사 미등록
이승철
곡 id: 3130464
제목: 이순간을 언제까지나
아티스트: 이승철
카라
곡 id: 3544237
제목: Strawberry
아티스트: 카라
조관우
곡 id: 3548838
제목: 달의 몰락 (김현철)
아티스트: 조관우
SUPER JUNIOR (슈퍼주니어)
곡 id: 3552393
제목: 엉뚱한 상상 (White Christmas)
아티스트: SUPER JUNIOR (슈퍼주니어)
티아라
곡 id: 3607862
제목: I'm So Bad
아티스트: 티아라
보아 (BoA)
곡 id: 3859897
제목: Not Over U
아티스트: 보아 (BoA)
f(x)
곡 id: 3874505
제목: NU 예삐오 (NU ABO)
아티스트: f(x)
리듬파워, 나인뮤지스
곡 id: 3879853
제목: Get Up
아티스트: 리듬파워
춘자 (春子)
곡 id: 3899012
제목: 오늘밤 (Feat. 남사장, 덩치)
아티스트: 춘자 (春子)
걸스데이
곡 id: 4054991
제목: Oh! My God!
아티스트: 걸스데이
주비스
곡 id: 4129024
제목: 사랑병 (Feat. 백민혁)
아티스트: 주비스
Dis Boyz
곡 id: 4196308
제목: 꿀꺽 (Feat. 아영 Of 달샤벳)
아티스트: Dis Boyz
더블에이
곡 id: 4209676
제목: 2nite 3x
아티스트: 더블에이
가사 미등록
SUPER JUNIOR (슈퍼주니어)
곡 id: 4853296
제목: 춤을 춘다 (Midnight Blues)
아티스트: SUPER JUNIOR (슈퍼주니